# Chapter 9 - Application Configuration

- Store secrets locally using the secrets manager and use the Azure Key Vault provider in other envrionments

    * https://github.com/dotnet/AspNetCore.Docs/blob/main/aspnetcore/security/app-secrets.md
    * https://learn.microsoft.com/en-us/aspnet/core/security/app-secrets?view=aspnetcore-9.0&tabs=windows

## Order settings are loaded

1. appsettings.json
2. appsettings.{env}.json
3. User secrets (Dev only)
4. Envrionment variables
5. CLI args



In [2]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Hosting.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Mvc.ViewFeatures.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Diagnostics.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.Results.dll"

I am passing settings since I do not have a appsettings.json.

```json
{
    "MyOptions": {
        "name": "Fred"
    }
}
```

In [21]:
string[] args = {"--urls","http://localhost:7001","--MyOptions:Name=Fred"};

This allows us to define our config with types and have them injected via the DI container.

In [23]:

using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Options;

var builder = WebApplication.CreateBuilder(args);
var myOptions = builder.Configuration.GetSection("MyOptions");

builder.Services.Configure<MyOptions>(myOptions);

var app = builder.Build();

app.MapGet("/", (IOptions<MyOptions> options) => options);

app.RunAsync();

public class MyOptions
{
    public string? Name { get; set; }
}

info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:7001
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\jason\training\dotnet\Architecting-ASP.NET-Core-Applications-3E\MyNotes


In [22]:
app.StopAsync();

In [29]:
using System.Net.Http;

var httpClient = new HttpClient();

var response = await httpClient.GetAsync("http://localhost:7001/");
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();
content.DisplayAs("application/json")

### Reload

- Can watch for changes to config using `IOptionsMonitor`

### Validation

-Can validate options using data annotations

```csharp
private class Options
{
    [Required]
    public string? MyImportantProperty { get; set; }
}
...
services.AddOptions<Options>()
    .Configure(o => o.MyImportantProperty = "A value")
    .ValidateDataAnnotations()
    .ValidateOnStart() // eager validation 
```

- Can also use custom validation for more complex logic

- He perfers to define his own class and use it instead the IOptions so he can set the lifetime

## Centralization

- All in one options + validation
- We trade off SRP for ease of management in this case

In [3]:
using Microsoft.Extensions.Options;

public class ProxyOptions : IConfigureOptions<ProxyOptions>, IValidateOptions<ProxyOptions>
{
    public static readonly int DefaultCacheTimeInSeconds = 60;

    public string? Name { get; set; }
    public int CacheTimeInSeconds { get; set; }

    void IConfigureOptions<ProxyOptions>.Configure(ProxyOptions options)
    {
        options.CacheTimeInSeconds = DefaultCacheTimeInSeconds;
    }

    ValidateOptionsResult IValidateOptions<ProxyOptions>.Validate(string? name, ProxyOptions options)
    {
        if (string.IsNullOrWhiteSpace(options.Name))
        {
            return ValidateOptionsResult.Fail("The 'Name' property is required.");
        }
        return ValidateOptionsResult.Success;
    }
}


Trying to start the app below without the Name option fails with:

```
fail: Microsoft.Extensions.Hosting.Internal.Host[11]
      Hosting failed to start
      Microsoft.Extensions.Options.OptionsValidationException: The 'Name' property is required.
         at Microsoft.Extensions.Options.OptionsFactory`1.Create(String name)
```

In [18]:
string[] args = {"--urls","http://localhost:7002","--myOptions:CacheTimeInSeconds=120"};

Here we can start it by passing the required `Name` option.

In [13]:
string[] args = {"--urls","http://localhost:7002","--myOptions:CacheTimeInSeconds=120","--myOptions:Name=MyOptionsExample"};

In [ ]:

using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Options;

var builder = WebApplication.CreateBuilder(args);
//We could encapsulate this in an extension method and just call `services.AddProxyService()`
var myOptions = builder.Configuration.GetSection("MyOptions");
builder.Services
    .AddSingleton<IConfigureOptions<ProxyOptions>, ProxyOptions>()
    .AddSingleton<IValidateOptions<ProxyOptions>, ProxyOptions>()
    .AddSingleton(sp => sp.GetRequiredService<IOptions<ProxyOptions>>().Value)
    .Configure<ProxyOptions>(myOptions)
    .AddOptions<ProxyOptions>()
    .ValidateOnStart()
;

var app = builder.Build();

app.MapGet("/", (ProxyOptions options) => options);

app.RunAsync();

fail: Microsoft.Extensions.Hosting.Internal.Host[11]
      Hosting failed to start
      Microsoft.Extensions.Options.OptionsValidationException: The 'Name' property is required.
         at Microsoft.Extensions.Options.OptionsFactory`1.Create(String name)
         at System.Lazy`1.ViaFactory(LazyThreadSafetyMode mode)
         at System.Lazy`1.ExecutionAndPublication(LazyHelper executionAndPublication, Boolean useDefaultConstructor)
         at System.Lazy`1.CreateValue()
         at Microsoft.Extensions.Options.OptionsCache`1.GetOrAdd[TArg](String name, Func`3 createOptions, TArg factoryArgument)
         at Microsoft.Extensions.Options.OptionsMonitor`1.Get(String name)
         at Microsoft.Extensions.DependencyInjection.OptionsBuilderExtensions.<>c__DisplayClass0_1`1.<ValidateOnStart>b__1()
         at Microsoft.Extensions.Options.StartupValidator.Validate()
      --- End of stack trace from previous location ---
         at Microsoft.Extensions.Options.StartupValidator.Validate()


In [16]:
using System.Net.Http;

var httpClient = new HttpClient();

var response = await httpClient.GetAsync("http://localhost:7002/");
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();
content.DisplayAs("application/json")

In [17]:
app.StopAsync();

## ValidateOptionResultBuilder class

> alidateOptionsResultBuilder is a new type in .NET 8. It allows us to dynamically accumulate validation errors and create a ValidateOptionsResult object representing its current state.

